In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from dataclasses import dataclass
from math import radians, pi, sin
import time

import numpy as np
import pandas as pd
import numba as nb
import matplotlib.pyplot as plt
from tqdm.auto import trange
import ipywidgets as widgets
import xarray.ufuncs as xrf
from IPython.display import display, JSON

from cw.context import time_it
from cw.filters import smooth_signal

from topone.environments.launcher_v1 import LauncherV1, Stage, Simulation, AP_NONE, AP_FLIGHT_PATH_CONTROL, AP_PITCH_CONTROL, AP_PITCH_RATE_CONTROL
from topone.environments.integrators import AB3Integrator
from topone.logger import Logger

# from environment import Environment
# from actor_critic_agent import ActorCriticAgent

from enum import Enum

tau = 2 * pi
hpi = pi / 2


In [3]:
logger = Logger()
env = LauncherV1(
    dt=0.01,
    surface_diameter=1737.4e3,
    mu=4.9048695e12,
    stages=(
        Stage(
            dry_mass=1,
            propellant_mass=0.02,
            specific_impulse=100,
            thrust=2*1.7),
    ),
    initial_altitude=1000,
    initial_theta_e=radians(90),
    initial_longitude=radians(90),
    gamma_controller_gains=(4, 0, 0.2),
    theta_controller_gains=(10, 0, 0.0),
)
logger.register_time_attribute(env.sim, "t")
logger.register(env.sim, "env", [
    "h",
    "gamma_e", "theta_e", "theta_i_dot",
    "ap_comm_gamma_e", "ap_comm_theta_e",
    "action_autopilot_mode", "action_autopilot_reference"])



In [4]:
with time_it("sim"):
    for j in trange(1):
        env.reset()
        for i in range(1000):
            if env.sim.t < 0.5:
                action = (True, False, AP_NONE, 0.0)
            else:
                action = (True, False, AP_FLIGHT_PATH_CONTROL, hpi + 0.1 * sin(env.sim.t * tau * 0.2))
            env.step(action)
            logger.step()
        episode_result = logger.episode_finish()

batch_results = logger.batch_finish()


sim: 2.272169068999574 [s]


In [5]:
plt.figure()
episode_result.env_gamma_e.plot()
episode_result.env_ap_comm_gamma_e.plot()

plt.figure()
episode_result.env_theta_e.plot()
episode_result.env_ap_comm_theta_e.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
episode_result.env_gamma_e

<xarray.DataArray 'env_gamma_e' (t: 1000)>
array([-1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633,
        1.57079633,  1.57079633,  1.55714112,  1.5593436 ,  1.54741606,
        1.53417526,  1.52820961,  1.5123208 ,  1.49952502,  1.48818635,
        1.47208207,  1.45938015,  1.44604179,  1.43149794,  1.41934434,
        1.40657292,  1.39438305,  1.38372008,  1.37310695,  1.36370176,
        1.35539572,  1.34771396,  1.34130395,  1.33583782,  1.33127022,
        1.32782526,  1.32524742,  1.32358561,  1.3228407 ,  1.32286316,
        1.32367097,  1.32518804,  1.32732886,  1.33006669,  1.333318  ,
        1.33702289,  1.3411307 ,  1.34557079,  1.35029106,  1.3552362 ,
        1.36034897,  1.36558127,  1.37088273,  1.37620643,  1.38150954,
        1.38674984,  1.391889  ,  1.39689136,  1.4017236 ,  1.40635588,
...
        1.39889555,  1.39887859,  1.39886257,  1.39884751,  1.3988334 ,
        1.39882027,  1.39880811,  1.39879692,  1.39878673,  1.39877752,
        1.3987693 ,  1.39876207,  1.39875584,  1.39875059,  1.39874635,
        1.39874309,  1.39874082,  1.39873953,  1.39873923,  1.3987399 ,
        1.39874155,  1.39874416,  1.39874772,  1.39875224,  1.39875769,
        1.39876408,  1.39877138,  1.3987796 ,  1.39878872,  1.39879872,
        1.3988096 ,  1.39882133,  1.39883391,  1.39884732,  1.39886155,
        1.39887658,  1.39889239,  1.39890896,  1.39892629,  1.39894434,
        1.39896311,  1.39898257,  1.3990027 ,  1.39902349,  1.39904491,
        1.39906694,  1.39908956,  1.39911275,  1.39913649,  1.39916075,
        1.39918552,  1.39921077,  1.39923647,  1.39926261,  1.39928916,
        1.3993161 ,  1.3993434 ,  1.39937104,  1.399399  ,  1.39942725,
        1.39945576,  1.39948452,  1.3995135 ,  1.39954267,  1.39957201,
        1.3996015 ,  1.3996311 ,  1.39966081,  1.39969058,  1.39972041,
        1.39975026,  1.39978011,  1.39980993,  1.39983971,  1.39986942,
        1.39989904,  1.39992853,  1.39995789,  1.39998709,  1.4000161 ,
        1.4000449 ,  1.40007348,  1.40010181,  1.40012986,  1.40015763,
        1.40018508,  1.4002122 ,  1.40023897,  1.40026536,  1.40029137,
        1.40031697,  1.40034214,  1.40036686,  1.40039113,  1.40041492,
        1.40043821,  1.40046099,  1.40048325,  1.40050496,  1.40052613])
Coordinates:
  * t        (t) float64 0.01 0.02 0.03 0.04 0.05 ... 9.96 9.97 9.98 9.99 10.0

In [35]:
result = logger.log_to_dataset(logger.raw_episode_log, True)
result

<xarray.Dataset>
Dimensions:         (d_2_0: 2, t: 1000)
Coordinates:
  * t               (t) float64 0.01 0.02 0.03 0.04 0.05 ... 9.97 9.98 9.99 10.0
Dimensions without coordinates: d_2_0
Data variables:
    env1_h          (t) float64 1e+03 1e+03 1e+03 ... 1.087e+03 1.087e+03
    env1_aii        (t, d_2_0) float64 1.047e-16 1.71 ... 1.119e-16 1.828
    env2_engine_on  (t) bool True True True True True ... True True True True
    env2_xii        (t, d_2_0) float64 1.065e-10 1.738e+06 ... 1.738e+06

In [33]:
plt.figure()
result.env1_h.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()
plt.plot(t_log[::100], h_log[::100], "-")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
JSON(env.sim_states_dict())

<IPython.core.display.JSON object>